# Machine Learning Project

In [ ]:
# Daniel Moskalchuk 293172
# Julia Tankiewicz 293719
# Nikita Bondarchuk 294478

The assignments below should be solved and documented as a project that will form the basis for the
examination. When solving the exercises it is important that you

  * document all relevant results and analyses that you have obtained/performed during the exercises.
  * try to relate your results to the theoretical background of the methods being applied.

Feel free to add cells if you need to.

Please hand in assignment 1-6 in a _**single**_ Jupyter notebook where you retain the questions outlined below. You are welcome to adapt code from the web (e.g. Kaggle kernels), but you **_must_** reference the original source in your notebook. In addition to _clean, well-documented code_ (i.e. functions with <a href="https://www.geeksforgeeks.org/python-docstrings/">docstrings</a>, etc), your notebook will be judged according to how well each step is explained (using Markdown). 

In general, direct questions regarding assignments 1, 4, 5 and 6 to Frederik, and questions regarding assignments 2, 3, and 7 to Richard. 

Last, but not least:
* Looking for an overview of the markdown language? The cheat sheet <a href="https://medium.com/ibm-data-science-experience/markdown-for-jupyter-notebooks-cheatsheet-386c05aeebed">here</a> might help.
* For the Python specific components of the exercises, you should not need constructs beyond those that are already included in the notebooks on the course's web-page (still you should not feel constrained by these, so feel free to be adventurous). You may, however, need to consult the documentation for some of the methods supplied by `sklearn`.

**Groups:** Create your own groups. May be across teams. 2-4 students per group. No one-person groups.


**Submission deadline:** Thursday, December 15 before 13.00 CET (Notebooks + presentation recording)

**Expected workload:** Each student is expected to spend around around 50 hours on the project.

### Deliverables
The teams have to submit three deliverables before the submission deadline: 1) a notebook of assignments 1-6, 2) a notebook of assignment 7, and 3) presentation video uploaded to some online platform e.g. YouTube, Vimeo, etc.

#### Notebook
The notebook contains all the code to explore the dataset, train the final model and documents each step clearly. If code is copied from another codebase such as Github or Stack Overflow it **_must_** be properly referenced.


#### Presentation
The presentation video should be 15 min long and should highlight the problem you are solving, interesting things you found in the data and the step involved in building up your model. At the exam we will discuss the presentation and ask questions about your project and submissions. A link to the video must be placed in the notebook for assignment 7.

### Randomness
For ALL random states, choose state = 69 so we can replicate your work.


In [ ]:
# Import all necessary modules here:
#To use following packages, pip install them first 
#pip install opencv-python
#pip install keras
#pip install --no-cache-dir tensorflow
#pip install shapely
#pip install geopandas
#pip install graphviz
#pip install pydotplus
#pip install tensorflow

from pathlib import Path
import numpy as np
import ast
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

from sklearn.tree import DecisionTreeClassifier
import graphviz
from sklearn.tree import export_graphviz
import pydotplus

from sklearn import (decomposition, ensemble, metrics)


#from sklearn import (neighbors, datasets, decomposition, ensemble,metrics, model_selection, preprocessing, linear_model)
#from keras.preprocessing import image
#from tensorflow.keras.utils import load_img, img_to_array


## 1. The IceCat Dataset

__You should be able to do this exercise after Lecture 3.__

The IceCat Dataset, kindly provided to us by Stibo Systems, contains a large amount of data on different office products. As an example of "real-world" data, these data are imperfect and incomplete. As such, this exercise is not so much an exercise in creating a good machine learning model, but places a larger emphasis on "cleaning the data".

We are going to work with a subset of the IceCat Dataset. In particular, you will be provided with a zip file of 5,854 images of office products, each with the name "product ID".jpg. You will also be provided with a list of colors, `colors.txt`, which, when imported using the code below, is a list of tuples of the form `[("product ID", "color"), ...]`. (The code below assumes that `colors.txt` is in the same folder as the jupyter notebook. Feel free to change the code if you prefer a different organization of your files).

Your task is to clean up the data and construct a simple machine learning model (_e.g._, _k_-nearest neighbor) that can identify the color of a product. You have free hands - there is hardly any one "correct answer" - but you need to argue for your choices. Among other things, you probably need to think about the following as you work with the data:

* All of the images have different sizes.

* Some of the images are RGB images (3 layers), others are CMYK (4 layers), some might even be black-and-white (1 layer).

* Some colors are only represented by very few products.

* Some colors are very similar, such as "Purple" and "Violet".

* A product may have a particular color, but a packaging of a different color. Similarly, the color of, say, a computer monitor may be black, while the image of it could show a monitor that is turned on with a green screensaver.

* Many products are attributed to several colors, such as "Black, Blue" or even "Blue, Green, Orange, Violet, Yellow". Yet others are described as "Multicolor" or "Assorted colors".

Again, you have free hands in how you are going to solve these (and other) challenges, but you must argue for and reflect on your choices as you progress.


In [ ]:
color_mappings = {
  # source color => arr of colors associated with the source color
    'Grey' : ['Silver', 'Metallic', 'Aluminium', 'Stainless steel', 'Platinum', 'Graphite', 'Chrome', 'Anthracite','Light Grey','Charcoal','Titanium','Light grey'],
    'Purple' : ['Violet', 'Lavender', 'Fuchsia','Magenta'],
    'Transparent' : ['Translucent'],
    'Cream' : ['Beige', 'Sand', 'Tan','Cappuccino','Ivory'],
    'Brown' : ['Oak colour', 'Wood','Bronze'],
    'Cyan' : ['Turquoise', 'Aqua colour'],
    'White' : ['Pearl' ],
    'Pink' : ['Rose'],
    'Red' : ['Cherry', 'Bordeaux'],
    'Green' : ['Pine', 'Lime','Olive'],
    'Yellow' : ['Gold'],
    'Blue' : ['Navy'],
    'Multicolour' : ['Multi','Assorted colours']
}

inputDataMock = {
    'img_id': [],
    'red1': [], 'green1': [], 'blue1': [],
    'red2': [], 'green2': [], 'blue2': [],
    'red3': [], 'green3': [], 'blue3': [],
    'red4': [], 'green4': [], 'blue4': [],
    'red5': [], 'green5': [], 'blue5': [],
    'color': []
}

___---METHODS---___

In [ ]:
# returns the source color in response to the associated color using the color_mappings
def colors_binning(initial_color):
    for source_color in color_mappings:
        if initial_color in color_mappings[source_color]: return source_color;
    return initial_color;

In [ ]:
# returns a dataframe from colors.txt
def getCorrectAnswersDF():
    with open("datasets/1/colors.txt","r") as file:
        colors = ast.literal_eval(file.read())
        # lets create a dataframe
        d = {'pic_id': [], 'color': []}
        for row in colors:
            # check if there are more colors in one row
            colors_arr = row[1].split(", ")
            d['pic_id'].append(row[0])
            d['color'].append(colors_binning(colors_arr[0])) 

        pic_colors = pd.DataFrame(data=d)
        return pic_colors;

In [ ]:
def getImgDataSorted(imgData, ignoreWhite = True, colorRound = 2, whiteThreshold = 0.15):
    pixel_arr = {}
    white = 1 - whiteThreshold;
    # itearate over images rows
    for imgRow in imgData:
        # iterate over pixels
        for pixelData in imgRow:
            red = round(pixelData.item(0) / 255, colorRound)
            green = round(pixelData.item(1) / 255, colorRound)
            blue = round(pixelData.item(2) / 255, colorRound)
            
            # skip white pixels
            if (ignoreWhite and red >= white and green >= white and blue >= white): 
                continue;
                
            key = str(red) +' '+ str(green) +' '+ str(blue);
            # if this pixel color is found in pixel_arr
            if (pixel_arr.get(key)): 
                pixel_arr[key]['count'] += 1 # add 1 to the counter
            else: # insert the color, set the counter to 1
                pixel_arr[key] = {'R': red, 'G': green, 'B': blue, 'count': 1};
    # sort the pixels by number of occurencies and return
    return sorted(pixel_arr.values(), key=lambda x: x['count'], reverse=True)

def PopulateData(pic_id, pixel_arr, correctAnswer):      
    # move the data to the dataFrame mock
    inputDataMock['img_id'].append(pic_id)
    
    for i in range(1,6):
        try:
            color = pixel_arr[i-1]
        except IndexError: # Index out of range - no more colors found
            color = {'R': 1.0, 'G': 1.0, 'B': 1.0}
            
        inputDataMock['red'+str(i)].append(color['R'])
        inputDataMock['green'+str(i)].append(color['G'])
        inputDataMock['blue'+str(i)].append(color['B'])
    
    for row_id, row in correctAnswer.iterrows():
        col = str(row['color']).strip()
        if len(col) == 0: col = None;
        
        inputDataMock['color'].append(col)
        break;

In [ ]:
def testAccuracy(X_train, y_train, X_test, y_test):#lets see how accuracy depends on number of neighbours
    #training_accuracy = []
    #test_accuracy = []
    diff_arr = []
    # try n_neighbors from 1 to 10
    neighbors_settings = range(3, 25)

    for n_neighbors in neighbors_settings:
        # build the model
        clf = KNeighborsClassifier(n_neighbors=n_neighbors)
        clf.fit(X_train, y_train)
        # record training set accuracy
        train = clf.score(X_train, y_train)
        test = clf.score(X_test, y_test)
        diff = (train - test) / ((train + test) * 0.5)
        #training_accuracy.append(train)
        # record generalization accuracy
        #test_accuracy.append(test)
        diff_arr.append(diff)

    #plt.plot(neighbors_settings, training_accuracy, label="training accuracy")
    #plt.plot(neighbors_settings, test_accuracy, label="test accuracy")
    plt.plot(neighbors_settings, diff_arr, label="diff")
    plt.ylabel("Accuracy")
    plt.xlabel("n_neighbors")
    plt.legend(); 

In [ ]:
def resize_img(img, dimension=(64, 64)):
    img_resized = cv.resize(img, dimension, interpolation=cv.INTER_AREA)
    return img_resized

def normalize_img(img):
    if img.shape == (64, 64):
        img = cv.cvtColor(img.astype('float32'),cv.COLOR_GRAY2RGB)
    elif img.shape == (64, 64, 4):
        img = cv.cvtColor(img.astype('float32'),cv.COLOR_RGBA2RGB)

    return img

def compress_img(img, nr_of_colors = 16):
    
    # Lets compress the image using clustering
    # https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_ml/py_kmeans/py_kmeans_opencv/py_kmeans_opencv.html
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    flags = cv.KMEANS_PP_CENTERS
    compactness, labels, centers = cv.kmeans(np.float32(img.reshape((-1, 3))),
                                    nr_of_colors, None, criteria, 10, flags)
    centers = np.uint8(centers)
    new_colors = centers[labels.flatten()]
    img_noramlized = new_colors.reshape(img.shape)
    return img_noramlized
    

In [ ]:
# MAIN
def main(container_path, neigbours = 18, limit = 0):
    image_dir = Path(container_path)
    count = 0
    img_hash = {}
    uniq_img = []
    
    # first of all - correct answers
    correct_answers_DF = getCorrectAnswersDF();
    
    for file in image_dir.iterdir():

        img = plt.imread(file)
        
        img_resized = resize_img(img)
        
        count+=1;
        if limit > 0 and count > limit: break;
        
        img_arr = img_resized.tolist()
        if img_arr in uniq_img: continue;
        
        pic_id = file.stem
        
        uniq_img.append(img_arr)
        
        img_normalized = normalize_img(img_resized)
        
        img_compressed = compress_img(img_normalized)
        
        img_pixels_colors = getImgDataSorted(img_compressed)

        right_answerDF = correct_answers_DF[correct_answers_DF['pic_id'] == int(pic_id)]
        
        PopulateData(pic_id, img_pixels_colors, right_answerDF)
        
        #plt.title(file.stem)
        #plt.imshow(img_normalized)  
    
    inputData = pd.DataFrame(data=inputDataMock)
    
    inputData = inputData.drop(columns = ['red4', 'green4', 'blue4', 'red5', 'green5', 'blue5']) #
    
    #inputData = pd.unique(inputData)
    
    color_counts = inputData['color'].value_counts(ascending=True)

    for color, count in color_counts.items():
        if(count < 2): inputData.drop(inputData[inputData['color'] == color ].index, inplace=True)
        else: break; 
        
    display(inputData)
    
    X = inputData.drop(columns = ['img_id', 'color'])
    Y = inputData['color']
    
    # split the data in training and test set
    X_train, X_test, y_train, y_test = train_test_split(
    X, Y, stratify=Y, random_state=69)
    
    knn = KNeighborsClassifier(n_neighbors=neigbours)
    knn.fit(X_train, y_train)
    print("Train set score of "+str(neigbours)+"-nn: {:.2f}".format(knn.score(X_train, y_train)))
    print("Test set score of "+str(neigbours)+"-nn: {:.2f}".format(knn.score(X_test, y_test)))
    
    #testAccuracy(X_train, y_train, X_test, y_test)
            
main("datasets/1/images");

## 2. Flights Departing from NYC

__You should be able to do this exercise after Lecture 4.__

For this exercise we will be using the famous nycflights13 data which contains the `airlines`, `airports`, `flights`, `planes`, and `weather` datasets. Please see the documentation (`nycflights13.pdf`) for further information.

**(a)** Load all files as pandas dataframes and display the first 5 rows of each dataset.

In [ ]:
#JULIA
#loading
airlines = pd.read_csv('datasets/2/airlines.csv')
airports = pd.read_csv('datasets/2/airports.csv')
flights = pd.read_csv('datasets/2/flights.csv')
planes = pd.read_csv('datasets/2/planes.csv')
weather = pd.read_csv('datasets/2/weather.csv')
#displaying
display(airlines.head())
display(airports.head())
display(flights.head())
display(planes.head())
display(weather.head())

**(b)** Convert all temperature attributes to degree Celsius. We will be using this in what follows.

In [ ]:
#JULIA
weather['temp'] = (weather['temp']-32)*5/9
weather['temp']

**(c)** Using OLS, investigate if flight distance is associated with arrival delay. You should be cautious regarding negative delays.

In [ ]:
#JULIA

positive_delays = flights[flights['arr_delay']>0]
negative_delays = flights[flights['arr_delay']<0]

def plotRegression(feature,df):
    df = df.dropna(subset=[feature,'arr_delay'])
    X = df[[feature]]
    y = df['arr_delay']
    X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)
    ODS = LinearRegression()
    ODS.fit(X_train, y_train)

    print("R^2 on train data is {} and on test data is {}".format(ODS.score(X_train, y_train),ODS.score(X_test,y_test)))
    print("The coefficents are {} and the intercept is {}".format(ODS.coef_, ODS.intercept_))
    print(df[['arr_delay',feature]].corr())
    
    y_pred = ODS.predict(X)
    plt.scatter(X, y,color = "green") 
    plt.plot(X[feature], y_pred,color = "blue")
    plt.xlabel(feature)
    plt.ylabel("Delay")
    plt.show()

plotRegression('distance',flights) #All fligths 
plotRegression('distance',positive_delays)#flights with only positive delays
plotRegression('distance',negative_delays)#flights with only negative delays


### Conclusions
According to graphs above the arrival delay and distance are correlated almost only in case of negative delays. That means that flights flying further distances were deleyed less(so they arrived sooner) than flights flying short distances, but that's true mostly only for when the flights departed ealier than scheduled(3rd graph). For flights with regular(positive) delays there's hardly any correlation between delay time and distance(2nd graph). Concluding, there's very little correlation between two features and it's mostly influenced by negative delays. 

**(d)** Using OLS, investigate if departure delay is associated with arrival delay. Again,
   consider what to do with negative delays.

In [ ]:
#JULIA

positive_delays = flights[((flights['dep_delay']>0) & (flights['arr_delay']>0))]
negative_delays = flights[((flights['dep_delay']<0) & (flights['arr_delay']<0))]

def plotRegression(feature,df):
    df = df.dropna(subset=[feature,'arr_delay'])
    X = df[[feature]]
    y = df['arr_delay']
    X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)
    ODS = LinearRegression()
    ODS.fit(X_train, y_train)

    print("R^2 on train data is {} and on test data is {}".format(ODS.score(X_train, y_train),ODS.score(X_test,y_test)))
    print("The coefficents are {} and the intercept is {}".format(ODS.coef_, ODS.intercept_))
    print(df[['arr_delay',feature]].corr())
    
    y_pred = ODS.predict(X)
    plt.scatter(X, y,color = "green") 
    plt.plot(X[feature], y_pred,color = "blue")
    plt.xlabel(feature)
    plt.ylabel("Delay")
    plt.show()

plotRegression('dep_delay',flights) #All fligths 
plotRegression('dep_delay',positive_delays)#flights with only positive delays
plotRegression('dep_delay',negative_delays)#flights with only negative delays



### Conclusions
The correlation between departure and arrival time is big in case of late flights but little in case of fligts that departed before expected time. So the correlations between these two is high but lowered by negative delays where arrival delay doesn't seem to be influenced by how early(earlier than expected time) the flight departed. It could be explained by fact that if flight departed ealier than expected it won't rush and usually land in planned time as it's probably scheduled and airports might not often allow early arrivals(it's just hypothesis :)). 

**(e)** Investigate whether departure delay is associated with weather conditions
   at the origin airport. This includes descriptives, plotting, regression modelling,
   considering missing values etc. For regression, do OLS, Ridge, Lasso, and Elastic Net.
   The analysis should also include seasonality trends as a "weather condition". You could,
   for instance, plot the daily departure delay with the date (or monthly). What are the
   three most important weather conditions when trying to predict departure delays?

In [ ]:
#JULIA
display(flights.head())
display(weather.head())
#remove flights with no hour data - unable to be merged with weather data
flights_complete = flights.dropna(subset=['hour'])
#remove unrelated features
flights_complete = flights_complete.drop(columns=['carrier', 'tailnum','flight','dest','air_time','distance','arr_delay','arr_time','minute'])
flights_complete.info()
weather.info()

scaler = MinMaxScaler()

In [ ]:
#merging weather and flights data and leaving only ones with departure delay non-nan values
flight_weather = pd.merge(flights_complete, weather, how='outer', on=['origin', 'year','month','day','hour'])
flight_weather = flight_weather.drop(columns=['origin','time_hour','year'])
flight_weather = flight_weather.dropna(subset=['dep_delay'])


#flight_weather[feature] = scaler.fit_transform(flight_weather[[feature]])

flight_weather.head()
flight_weather.info()
flight_weather.describe()

flight_weather = flight_weather[flight_weather["wind_speed"] < 1000] #there's outlier that is probably error, because it's impossible for wind speed to be above 1000mph(The Highest Anemometer-Measured Wind Speeds on Earth, 2022)

In [ ]:
#now trying to get rid of missing value I'm using regression imputing for it
missing_columns = ["temp", "dewp", "humid", "wind_dir", "wind_speed","wind_gust","precip","pressure","visib"]
#firstly random impute missing values - we have multiple variables with missing values so some predictors woud be nan and it wouldn't work
def random_imputation(df, feature):

    number_missing = df[feature].isnull().sum()
    observed_values = df.loc[df[feature].notnull(), feature]
    df.loc[df[feature].isnull(), feature + '_imp'] = np.random.choice(observed_values, number_missing, replace = True)
    
    return df

for feature in missing_columns:
    flight_weather[feature + '_imp'] = flight_weather[feature]
    flight_weather = random_imputation(flight_weather, feature)
    
deter_data = pd.DataFrame(columns = ["Det" + name for name in missing_columns])

#using linear regression to input all the random imputed values with result of lin. regression for each feature with missing values
for feature in missing_columns:
        
    deter_data["Det" + feature] = flight_weather[feature + "_imp"]
    parameters = list(set(flight_weather.columns) - set(missing_columns) - {feature + '_imp'})
    model = LinearRegression()
    model.fit(X = flight_weather[parameters], y = flight_weather[feature + '_imp'])
    deter_data.loc[flight_weather[feature].isnull(), "Det" + feature] = model.predict(flight_weather[parameters])[flight_weather[feature].isnull()]
    flight_weather[feature] = deter_data["Det" + feature]
    flight_weather = flight_weather.drop(columns=[feature + "_imp"])
    

flight_weather.info()


## OLS

In [ ]:
    X = flight_weather.drop(columns=['dep_delay'])
    y = flight_weather['dep_delay']
    X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)
    ODS = LinearRegression()
    ODS.fit(X_train, y_train)

    print("R^2 on train data is {} and on test data is {}".format(ODS.score(X_train, y_train),ODS.score(X_test,y_test)))
    print("the intercept is {}".format(ODS.intercept_))
    pd.Series(ODS.coef_, index=X.columns)
    

#### Ploting OLS for each feature separately to see their correlation with dep delay time

In [ ]:
def plotRegression(feature,flight_weather):
    #flight_weather[feature] = scaler.fit_transform(flight_weather[[feature]])
    X = flight_weather[[feature]]
    y = flight_weather['dep_delay']
    X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)
    ODS = LinearRegression()
    ODS.fit(X_train, y_train)

    print("R^2 on train data is {} and on test data is {}".format(ODS.score(X_train, y_train),ODS.score(X_test,y_test)))
    print(flight_weather[['dep_delay',feature]].corr())
    
    y_pred = ODS.predict(X)
    plt.scatter(X, y,color = "green") 
    plt.plot(X[feature], y_pred,color = "blue")
    plt.xlabel(feature)
    plt.ylabel("Delay")
    plt.show()


In [ ]:
for feature in flight_weather.columns:
    plotRegression(feature,flight_weather)


In [ ]:
#Important to run this once only, if rerun needed then run data loading and preproccesing again
#the correlation for month might depend on which month we start counting from, so trying to plot it trying different months
flight_weather['month'] = flight_weather['month'] - 8
flight_weather[flight_weather['month']<0] = flight_weather[flight_weather['month']<0] + 12
plotRegression('month',flight_weather)
#same for hour
flight_weather['hour'] = flight_weather['hour'] - 5
flight_weather[flight_weather['hour']<0] = flight_weather[flight_weather['hour']<0] + 24
plotRegression('hour',flight_weather)

## Ridge



In [ ]:
alphas = 10**np.linspace(-10, 10, 100)

for feature in flight_weather.columns:
    flight_weather[feature] = scaler.fit_transform(flight_weather[[feature]])

X = flight_weather.drop(columns=['dep_delay'])
y = flight_weather['dep_delay']
    
ridge = Ridge(normalize = True)
coefs = []

for a in alphas:
    ridge.set_params(alpha = a)
    ridge.fit(X, y)
    coefs.append(ridge.coef_)
    
np.shape(coefs)
temp_coefficients = [coef[0] for coef in coefs]
dewp_coefficients = [coef[1] for coef in coefs]
humid_coefficients = [coef[2] for coef in coefs]
wind_dir_coefficients = [coef[3] for coef in coefs]
wind_speed_coefficients = [coef[4] for coef in coefs]
wind_gust_coefficients = [coef[5] for coef in coefs]
precip_coefficients = [coef[6] for coef in coefs]
pressure_coefficients = [coef[7] for coef in coefs]
visib_coefficients = [coef[8] for coef in coefs]
month_coefficients = [coef[9] for coef in coefs]
day_coefficients = [coef[10] for coef in coefs]

In [ ]:
ax = plt.gca()
ax.set_xscale('log')
ax.plot(alphas, temp_coefficients, label="temp coefficient")
ax.plot(alphas, dewp_coefficients, label="dewp coefficient")
ax.plot(alphas, humid_coefficients, label="humid coefficient")
ax.plot(alphas, wind_dir_coefficients, label="wind_dir coefficient")
ax.plot(alphas, wind_speed_coefficients, label="wind_speed coefficient")
ax.plot(alphas, wind_gust_coefficients, label="wind_gust coefficient")
ax.plot(alphas, precip_coefficients, label="precip coefficient")
ax.plot(alphas, pressure_coefficients, label="pressure coefficient")
ax.plot(alphas, visib_coefficients, label="visib coefficient")
ax.plot(alphas, month_coefficients, label="month coefficient")
ax.plot(alphas, day_coefficients, label="day coefficient")
plt.legend()
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')

In [ ]:
    #alphas = 10**np.linspace(10,-2,100)*0.5
    X = flight_weather.drop(columns=['dep_delay'])
    y = flight_weather['dep_delay']
    X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)

    ridgecv = RidgeCV( normalize = True)#alphas = alphas,
    ridgecv.fit(X_train, y_train)
    print(ridgecv.alpha_)   
    print("R^2 on train data is {} and on test data is {}  ".format(ridgecv.score(X_train, y_train), ridgecv.score(X_test,y_test)))
    pd.Series(ridgecv.coef_, index = X.columns)

## Lasso

In [ ]:
lasso = Lasso(max_iter = 10000, normalize = True)
X = flight_weather.drop(columns=['dep_delay'])
y = flight_weather['dep_delay']
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)
lassocv = LassoCV(cv = 10, max_iter = 100000, normalize = True)#alphas = None,
lassocv.fit(X_train, y_train)
print(lassocv.alpha_)
print("R^2 on train data is {} and on test data is {}".format(lassocv.score(X_train, y_train),lassocv.score(X_test,y_test)))
pd.Series(lassocv.coef_, index=X.columns)

#lasso.set_params(alpha=lassocv.alpha_)
#lasso.fit(X_train, y_train)

#print("R^2 on train data is {} and on test data is {}".format(lassocv.score(X_train, y_train),lassocv.score(X_test,y_test)))
#pd.Series(lasso.coef_, index=X.columns)

## Elastic Net

In [ ]:

    X = flight_weather.drop(columns=['dep_delay'])
    y = flight_weather['dep_delay']
    X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)
    #cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=69)
    ratios = [.1, .2, .3 , .4, .5, .7, .8, .9, .95, .99, 1]
    #alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0]
    en = ElasticNetCV(l1_ratio=ratios, cv=10, n_jobs=-1)
    en.fit(X_train, y_train)
    print('alpha: %f' % en.alpha_)
    print('l1_ratio_: %f' % en.l1_ratio_)
    print("R^2 on train data is {} and on test data is {}".format(en.score(X_train, y_train),en.score(X_test,y_test)))

## Conclusions
After building regression on weather data with different models it's visible that the most 'influencial' parameter is hour. Top three are: hour, month and precip(now it depends on model). Weather conditions don't really influence deperture delay much therefore it's hard to build good regression model on it.

**(f)** Is the age of the plane associated with delay? Do OLS, Ridge, Lasso, and Elastic Net.

In [ ]:
#NIKITA

# DATA PREPARATION 

planes_flights = pd.merge(planes, flights, how='outer', left_on = 'tailnum', right_on = 'tailnum')
planes_flights = planes_flights.drop(['type', 'manufacturer', 'seats', 'model', 'engines', 'speed', 'engine', 'carrier', 'flight', 'origin', 'dest', 'arr_time', 'dep_time', 'air_time', 'distance'], axis=1)
planes_flights['years'] = planes_flights['year_y']-planes_flights['year_x']
planes_flights['delay'] = planes_flights['dep_delay'] + (planes_flights['arr_delay'] - planes_flights['dep_delay'] )
planes_flights = planes_flights.drop(['arr_delay', 'dep_delay', 'year_x', 'year_y', 'month', 'day', 'hour', 'minute'], axis=1)

# taking average delay of each plane

planes_flights = planes_flights.groupby(['tailnum']).mean()
planes_flights = planes_flights.reset_index()
planes_flights = planes_flights.drop(['tailnum'], axis=1)

planes_flights = planes_flights.dropna(subset=['years'])

planes_flights = planes_flights.sort_values(by=['delay'])

#X = result['delay'].array.reshape(-1, 1)
#y = result['years']

#X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

In [ ]:
#now trying to get rid of missing value I'm using regression imputing for it
missing_columns = ["delay"]
#firstly random impute missing values - we have multiple variables with missing values so some predictors woud be nan and it wouldn't work
def random_imputation(df, feature):

    number_missing = df[feature].isnull().sum()
    observed_values = df.loc[df[feature].notnull(), feature]
    df.loc[df[feature].isnull(), feature + '_imp'] = np.random.choice(observed_values, number_missing, replace = True)
    
    return df

for feature in missing_columns:
    planes_flights[feature + '_imp'] = planes_flights[feature]
    planes_flights = random_imputation(planes_flights, feature)
    
deter_data = pd.DataFrame(columns = ["Det" + name for name in missing_columns])

#using linear regression to input all the random imputed values with result of lin. regression for each feature with missing values
for feature in missing_columns:
        
    deter_data["Det" + feature] = planes_flights[feature + "_imp"]
    parameters = list(set(planes_flights.columns) - set(missing_columns) - {feature + '_imp'})
    model = LinearRegression()
    model.fit(X = planes_flights[parameters], y = planes_flights[feature + '_imp'])
    deter_data.loc[planes_flights[feature].isnull(), "Det" + feature] = model.predict(planes_flights[parameters])[planes_flights[feature].isnull()]
    planes_flights[feature] = deter_data["Det" + feature]
    planes_flights = planes_flights.drop(columns=[feature + "_imp"])
    

planes_flights.info()

In [ ]:
# Correlation of years/delay

print(planes_flights[['years','delay']].corr())

In [ ]:
def plotRegression(X_train,y_train):
    plt.scatter(X_train, y_train, color='red') # plotting the observation line
    plt.plot(X_train, lr.predict(X_train), color='blue') # plotting the regression line
    plt.title("(Training set)") # stating the title of the graph
    plt.xlabel("Delay") # adding the name of x-axis
    plt.ylabel("Ages") # adding the name of y-axis
    plt.show() # specifies end of graph

In [ ]:
## OLS

X = planes_flights['delay'].array.reshape(-1, 1)
y = planes_flights['years']
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

lr = LinearRegression()
lr.fit(X_train, y_train)

print("lr.coef_: {}".format(lr.coef_))
print("lr.intercept_: {}".format(lr.intercept_))
print("Training set score (R2): {:.6f}".format(lr.score(X_train, y_train)))
print("Test set score: {:.6f}".format(lr.score(X_test, y_test)))
print('Difference in training and test set score signs of overfiting')

## making graph
plotRegression(X_train,y_train)

In [ ]:
#Ridge
alphas = 10**np.linspace(-10, 10, 100)
ridge = Ridge(normalize = True)
coefs = []

for a in alphas:
    ridge.set_params(alpha = a)
    ridge.fit(X_train, y_train)
    coefs.append(ridge.coef_)

np.shape(coefs)
_coefficients = [coef[0] for coef in coefs]

In [ ]:
ax = plt.gca()
ax.set_xscale('log')
ax.plot(alphas, _coefficients, label="delay coefficient")
plt.legend()
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')

In [ ]:
X = planes_flights['delay'].array.reshape(-1, 1)
y = planes_flights['years']
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

ridgecv = RidgeCV(normalize = True)#alphas = alphas,
ridgecv.fit(X_train, y_train)
print(ridgecv.alpha_)   
print("R^2 on train data is {} and on test data is {}  ".format(ridgecv.score(X_train, y_train), ridgecv.score(X_test,y_test)))

## making graph
plotRegression(X_train,y_train)

In [ ]:
#Lasso
X = planes_flights['delay'].array.reshape(-1, 1)
y = planes_flights['years']
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

#we increase the default setting of "max_iter",
#otherwise the model would warn us that we should increase max_iter.
lasso = Lasso(alpha=0.01, max_iter=10000, normalize = True).fit(X_train, y_train)
print("Training set score: {:.6f}".format(lasso.score(X_train, y_train)))
print("Test set score: {:.6f}".format(lasso.score(X_test, y_test)))
print("Number of features used: {}".format(np.sum(lasso.coef_ != 0)))

## making graph
plotRegression(X_train,y_train)

In [ ]:
#LassoCV
X = planes_flights['delay'].array.reshape(-1, 1)
y = planes_flights['years']
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

lassocv = LassoCV(cv = 10, max_iter = 100000, normalize = True)#alphas = None,
lassocv.fit(X_train, y_train)
print(lassocv.alpha_)
print("R^2 on train data is {} and on test data is {}".format(lassocv.score(X_train, y_train),lassocv.score(X_test,y_test)))

## making graph
plotRegression(X_train,y_train)

In [ ]:
# ElasticNet
X = planes_flights['delay'].array.reshape(-1, 1)
y = planes_flights['years']
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)
#cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=69)
ratios = [.1, .2, .3 , .4, .5, .7, .8, .9, .95, .99, 1]
#alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0]
en = ElasticNetCV(l1_ratio=ratios, cv=10, n_jobs=-1)
en.fit(X_train, y_train)

print('alpha: %f' % en.alpha_)
print('l1_ratio_: %f' % en.l1_ratio_)
print("R^2 on train data is {} and on test data is {}".format(en.score(X_train, y_train),en.score(X_test,y_test)))

## making graph
plotRegression(X_train,y_train)




**(g)** Do a principal component analysis of the weather at JFK using the following columns:
   temp, dewp, humid, wind_dir, wind_speed, precip, visib.
   How many principal components should be used to capture the variability in the weather data?

In [ ]:
# Nikita 
features = ['temp', 'dewp', 'humid', 'wind_dir', 'wind_speed', 'precip', 'visib']
weather_to_use = weather.dropna()

display(weather_to_use)

# Separating out the features
X = weather_to_use.loc[:, features].values
display(X)
# Separating out the origin
y = weather_to_use['origin'].values
display(y)
# Splitting the X and Y into the
sc = StandardScaler()
# Standardizing the features
sc.fit(X)
X_scaled = sc.transform(X)

# Applying PCA 
# 7 components sa the number of features we have
pca = decomposition.PCA(n_components = 7)
pca.fit(X_scaled)
X_pca_7 = pca.transform(X_scaled)

print("variance explained by all 7 principal components =", sum(pca.explained_variance_ratio_ * 100))

plt.plot(np.cumsum(pca.explained_variance_ratio_ * 100))
plt.xlabel('Number of components')
plt.ylabel('Explained variance')
plt.savefig('elbow_plot.png', dpi=100)

print("variance explained by the First principal component =", 
      np.cumsum(pca.explained_variance_ratio_ * 100)[0])

print("variance explained by the 2 principal component =", 
      np.cumsum(pca.explained_variance_ratio_ * 100)[1])

print("variance explained by the 3 principal component =", 
      np.cumsum(pca.explained_variance_ratio_ * 100)[2])

print("variance explained by the 4 principal component =", 
      np.cumsum(pca.explained_variance_ratio_ * 100)[3])

print("variance explained by the 5 principal component =", 
      np.cumsum(pca.explained_variance_ratio_ * 100)[4])

print("variance explained by the 6 principal component =", 
      np.cumsum(pca.explained_variance_ratio_ * 100)[5])

print("variance explained by the 7 principal component =", 
      np.cumsum(pca.explained_variance_ratio_ * 100)[6])

# It is not possible to create a scatterplot for the original datase because it contains 7 features.
# We can make 2D and 3D scatterplots, but in this case we will lose some variability
# For 2D scatterplot we will have 56.6...% variability
# For 3D scatterplot we will have 72.2...% variability 

# Variance after 5 porincipal componenet are not changing significantlty 

# Select the best number of principal components while keeping as much of the variance in the original data as possible.

# We can choose 5 prncipal components to capture the variability in the weather data


**(h)** Build regression models (OLS, Ridge, Lasso, and Elastic Net) that associates
   an airports lattitude with weather conditions (temp, dewp, humid, wind_dir, wind_speed,
   precip, visib). Remove all but the three most significant whether conditions and redo
   the analysis.

In [ ]:
# Nikita 
# Data Preparation
data = pd.merge(airports, weather, how='outer', left_on = 'faa', right_on = 'origin')
data = data.drop(columns = ['year', 'month', 'day', 'hour','lon', 'alt', 'tz', 'dst', 'tzone', 'origin', 'name', 'time_hour', 'pressure', 'wind_gust', 'faa'])
display(data)
data = data.dropna(subset=['lat'])
data = data[data["wind_speed"] < 1000]


In [ ]:
#now trying to get rid of missing value I'm using regression imputing for it
missing_columns = ["temp", "dewp", "humid", "wind_dir", "wind_speed","precip","visib"]
#firstly random impute missing values - we have multiple variables with missing values so some predictors woud be nan and it wouldn't work

for feature in missing_columns:
    data[feature + '_imp'] = data[feature]
    data = random_imputation(data, feature)
    
deter_data = pd.DataFrame(columns = ["Det" + name for name in missing_columns])

#using linear regression to input all the random imputed values with result of lin. regression for each feature with missing values
for feature in missing_columns:
        
    deter_data["Det" + feature] = data[feature + "_imp"]
    parameters = list(set(data.columns) - set(missing_columns) - {feature + '_imp'})
    model = LinearRegression()
    model.fit(X = data[parameters], y = data[feature + '_imp'])
    deter_data.loc[data[feature].isnull(), "Det" + feature] = model.predict(data[parameters])[data[feature].isnull()]
    data[feature] = deter_data["Det" + feature]
    data = data.drop(columns=[feature + "_imp"])
    

data.info()

In [ ]:
# OLS 

X = data.loc[:,'temp':'visib']
y = data['lat']
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)
lr = LinearRegression()
lr.fit(X_train, y_train)

print("R^2 on train data is {} and on test data is {}".format(lr.score(X_train, y_train),lr.score(X_test,y_test)))
print("the intercept is {}".format(lr.intercept_))
pd.Series(lr.coef_, index=X.columns)
    

In [ ]:
def plotOLSRegression(feature, data):
    X = data[[feature]]
    y = data['lat']
    X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    print("R^2 on train data is {} and on test data is {}".format(lr.score(X_train, y_train),lr.score(X_test,y_test)))
    print(data[['lat',feature]].corr())
    
    y_pred = lr.predict(X)
    plt.scatter(X,y,color = "green") 
    plt.plot(X[feature], y_pred,color = "blue")
    plt.xlabel(feature)
    plt.ylabel("Lat")
    plt.show()

In [ ]:
#OLS

for feature in data.columns:
    plotOLSRegression(feature,data)

In [ ]:
#Ridge

alphas = 10**np.linspace(-10, 10, 100)

X = data.loc[:,'temp':'visib']
y = data['lat']
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)
ridge = Ridge(normalize = True)
coefs = []

for a in alphas:
    ridge.set_params(alpha = a)
    ridge.fit(X_train, y_train)
    coefs.append(ridge.coef_)
    
np.shape(coefs)
temp_coefficients = [coef[0] for coef in coefs]
dewp_coefficients = [coef[1] for coef in coefs]
humid_coefficients = [coef[2] for coef in coefs]
wind_dir_coefficients = [coef[3] for coef in coefs]
wind_speed_coefficients = [coef[4] for coef in coefs]
precip_coefficients = [coef[5] for coef in coefs]
visib_coefficients = [coef[6] for coef in coefs]

In [ ]:
ax = plt.gca()
ax.set_xscale('log')
ax.plot(alphas, temp_coefficients, label="temp coefficient")
ax.plot(alphas, dewp_coefficients, label="dewp coefficient")
ax.plot(alphas, humid_coefficients, label="humid coefficient")
ax.plot(alphas, wind_dir_coefficients, label="wind_dir coefficient")
ax.plot(alphas, wind_speed_coefficients, label="wind_speed coefficient")
ax.plot(alphas, precip_coefficients, label="precip coefficient")
ax.plot(alphas, visib_coefficients, label="visib coefficient")
plt.legend()
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')

In [ ]:
#RidgeCV

X = data.loc[:,'temp':'visib']
y = data['lat']
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)

ridgecv = RidgeCV(normalize = True)
ridgecv.fit(X_train, y_train)
print(ridgecv.alpha_)   
print("R^2 on train data is {} and on test data is {}  ".format(ridgecv.score(X_train, y_train), ridgecv.score(X_test,y_test)))
pd.Series(ridgecv.coef_, index = X.columns)

In [ ]:
#Lasso

X = data.loc[:,'temp':'visib']
y = data['lat']
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)
lasso = Lasso(alpha = 8.3,max_iter = 10000, normalize = True)#alphas = None,
lasso.fit(X_train, y_train)
print("R^2 on train data is {} and on test data is {}".format(lasso.score(X_train, y_train),lasso.score(X_test,y_test)))

In [ ]:
#LassoCV

X = data.loc[:,'temp':'visib']
y = data['lat']
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)

lassocv = LassoCV(cv = 10, max_iter = 100000, normalize = True)#alphas = None,
lassocv.fit(X_train, y_train)
print(lassocv.alpha_)
print("R^2 on train data is {} and on test data is {}".format(lassocv.score(X_train, y_train),lassocv.score(X_test,y_test)))

pd.Series(lassocv.coef_, index=X.columns)

In [ ]:
# Elastic Net CV

X = data.loc[:,'temp':'visib']
y = data['lat']
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)

#cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=69)
ratios = [.1, .2, .3 , .4, .5, .7, .8, .9, .95, .99, 1]
#alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0]
en = ElasticNetCV(l1_ratio=ratios, cv=10, n_jobs=-1)
en.fit(X_train, y_train)
print('alpha: %f' % en.alpha_)
print('l1_ratio_: %f' % en.l1_ratio_)
print("R^2 on train data is {} and on test data is {}".format(en.score(X_train, y_train),en.score(X_test, y_test)))
pd.Series(en.coef_, index=X.columns)

In [ ]:
# Remove all but the three most significant whether conditions and redo the analysis.
# temp, dewp, humid, wind_dir, wind_speed, precip, visib

# dewp, precip, visib have more significant change after doing ElasticNetCV 

data_upd = data.drop(['temp', 'wind_speed', 'humid', 'wind_dir'], axis=1)

data_upd



X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)


In [ ]:
#OLS
X = data_upd.loc[:,'dewp':'visib']
y = data_upd['lat']
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)
lr = LinearRegression()
lr.fit(X_train, y_train)

print("R^2 on train data is {} and on test data is {}".format(lr.score(X_train, y_train),lr.score(X_test,y_test)))
print("the intercept is {}".format(lr.intercept_))
pd.Series(lr.coef_, index=X.columns)

In [ ]:
for feature in data_upd.columns:
    plotOLSRegression(feature,data_upd)

In [ ]:
#Ridge

X = data_upd.loc[:,'dewp':'visib']
y = data_upd['lat']
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)

ridgecv = RidgeCV(normalize = True)
ridgecv.fit(X_train, y_train)
print(ridgecv.alpha_)   
print("R^2 on train data is {} and on test data is {}  ".format(ridgecv.score(X_train, y_train), ridgecv.score(X_test,y_test)))
pd.Series(ridgecv.coef_, index = X.columns)


In [ ]:
#Lasso

X = data_upd.loc[:,'dewp':'visib']
y = data_upd['lat']
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)

lassocv = LassoCV(cv = 10, max_iter = 100000, normalize = True)#alphas = None,
lassocv.fit(X_train, y_train)
print(lassocv.alpha_)
print("R^2 on train data is {} and on test data is {}".format(lassocv.score(X_train, y_train),lassocv.score(X_test,y_test)))

pd.Series(lassocv.coef_, index=X.columns)


In [ ]:
# Elastic Net
X = data_upd.loc[:,'dewp':'visib']
y = data_upd['lat']
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=69)

#cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=69)
ratios = [.1, .2, .3 , .4, .5, .7, .8, .9, .95, .99, 1]
#alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0]
en = ElasticNetCV(l1_ratio=ratios, cv=10, n_jobs=-1)
en.fit(X_train, y_train)
print('alpha: %f' % en.alpha_)
print('l1_ratio_: %f' % en.l1_ratio_)
print("R^2 on train data is {} and on test data is {}".format(en.score(X_train, y_train),en.score(X_test, y_test)))
pd.Series(en.coef_, index=X.columns)



**(i)** On a map, plot the airports that have flights to them where the points that represent
   airports are relative in size to the average departure delay. You can see an example in "airports.png".

In [ ]:
# Nikita

data_map = pd.merge(airports, flights, how='outer', left_on = 'faa', right_on = 'dest')
data_map = data_map.dropna()

data_map = data_map.drop(['dep_time', 'arr_time', 'arr_delay', 'carrier', 'tailnum', 'flight', 'origin', 'dest', 'air_time', 'distance', 'tzone', 'tz', 'alt', 'name', 'faa', 'hour', 'minute', 'year', 'month', 'day', 'dst'], axis=1)

data_map = data_map.groupby(['lat', 'lon']).mean()

data_map = data_map.reset_index()

positive_delays = data_map[(data_map['dep_delay']>0)]

geometry = [Point(xy) for xy in zip(positive_delays['lon'], positive_delays['lat'])]
gdf = GeoDataFrame(positive_delays, geometry=geometry)   

#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', color='red', markersize=positive_delays['dep_delay'])





 **(j)** These questions require no code.
 - Suppose you are using Ridge Regression and you notice that the training error and the validation error are almost equal and fairly high. Would you say that the model suffers from high bias or high variance? Should you increase the regularization hyperparameter or reduce it?

- Why would you want to use:
        > Ridge Regression instead of plain Linear Regression (i.e. without any regularization)?
        > Lasso instead of Ridge Regression?
        > Elastic Net instead of Lasso?

In [ ]:
# Nikita
# Suppose you are using Ridge Regression and you notice that the training error and the validation 
# error are almost equal and fairly high. Would you say that the model suffers from high bias or high 
# variance? Should you increase the regularization hyperparameter or reduce it?

# When the training error and validation error are close to each other and high that means your model 
# is underfitting (i.e. it has high bias). You should try to reduce the regularization hyperparameter.

In [ ]:
#Why would you want to use:

#Ridge Regression instead of plain Linear Regression (i.e. without any regularization)?

#When you have features in your dataset that are highly linearly correlated with other features, 
#turns out linear models will be likely to overfit. Ridge Regression, avoids over fitting 
#by adding a penalty to models that have too large coefficients.

#Lasso instead of Ridge Regression?

#Lasso tends to do well if there are a small number of significant parameters and the others 
#are close to zero.

# Elastic Net instead of Lasso?

#Elastic Net combines feature elimination from Lasso and feature coefficient reduction from 
#the Ridge model to improve your model's predictions.


## 3. Clustering of Handwritten Digits

__You should be able to do this exercise after Lecture 5.__

This exercise will depart from the famous MNIST dataset, and we are exploring several clustering techniques with it.. This is a ".mat" file, in order to load this file in an ipynb you have to use loadmat() function from scipy.io. (replace my path).

In [ ]:
import cv2 as cv
import numpy as np
import sklearn.utils as utils
from scipy.io import loadmat
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold

mnist = loadmat('datasets/3/mnist-original')
mnist_data = mnist["data"].T
mnist_label = mnist["label"][0]
unique_labels = np.unique(mnist_label)

print("Number of datapoints: {}\n".format(mnist_data.shape[0]))
print("Number of features: {}\n".format(mnist_data.shape[1]))
print("List of labels: {}\n".format(unique_labels))

In [ ]:
# let's perform feature selection for each class separately
def find_indices(lst, condition):
    return [i for i, elem in enumerate(lst) if condition(elem)]

X = []
Y = []
for label in unique_labels:
    indeces_by_label = find_indices(mnist_label, lambda e: e == label)
    
    data_for_label = mnist_data[indeces_by_label]
    variance_thr = VarianceThreshold().fit(data_for_label)
    selection_mask = variance_thr.get_support()
    
    print("Applying selection mask for class", int(label), "this may take some time...")
    # go through each instance of this class
    for i in indeces_by_label:
        img = mnist_data[i]
        new_img = []
        # go through each pixel of img
        for idx, px in enumerate(img):
            if(selection_mask[idx]): # if pixel was selected by the selection mask
                new_img.append(round(px / 255, 4)) # scale the pixel and move on
            else:
                new_img.append(0) # clear the pixel
        X.append(np.array(new_img))
        Y.append(label)
        
X = np.array(X)
Y = np.array(Y)

In [ ]:
# Train-Test Split
X_train, X_test , y_train, y_test = train_test_split(
    X, Y, test_size=0.2, stratify=Y, random_state=69)

There are 70,000 images, and each image has 784 features. This is because each image is 28×28 pixels,
and each feature simply represents one pixel’s intensity, from 0 (white) to 255 (black). Let’s take a peek at one digit from the dataset. All you need to do is grab an instance’s feature vector, reshape it to a 28×28 array, and display it using Matplotlib’s `imshow()` function:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
index = 123
print("Label of datapoint =", int(y_train[index]))
print("Value of datapoint:\n", X_train[index])
print("As image:\n")
plt.imshow(X_train[index].reshape(28,28), cmap=plt.cm.gray_r)
plt.show()

**(a)** Perform k-means clustering with k=10 on this dataset.

In [ ]:
raw_clusters = KMeans(n_clusters=10, random_state=69).fit(X_train)

accuracy = accuracy_score(y_train, raw_clusters.labels_);
print(
    "The accuracy is:",
    accuracy,
)

In [ ]:
index = 0
print("True label of datapoint =", int(y_train[index]))
print("Predicted label of datapoint =", int(raw_clusters.labels_[index]))
print("As image:\n")
plt.imshow(X_train[index].reshape(28,28), cmap=plt.cm.gray_r)
plt.show()

The accuracy is very low, so let's perform some feature selection. Let's remove the features that do not have much variance througout the data. To find the best Variance Threshold, let's go over a small part of the data with a loop


In [ ]:
best_accuracy = 0
i = 0
best_threshold = i
best_clusters = 0;
selected_features = []
selection_mask = [];

while(True):
    try:
        variance_thr = VarianceThreshold(i).fit(X_train)
        x = variance_thr.fit_transform(X_train)
        clusters = KMeans(n_clusters=10, random_state=69).fit(x)
    except: # it throuws an error when the threshold is too large for the data
        break

    accuracy = accuracy_score(y_train, clusters.labels_);
    
    if(accuracy > best_accuracy):
        best_accuracy = accuracy
        best_threshold = i
        best_clusters = clusters
        selection_mask = variance_thr.get_support()
        selected_features = x
    
    i = round(i + 0.01, 2)
    
print(
    "The best accuracy of", best_accuracy,
    "resulted from the threshold of", best_threshold,  
)

In [ ]:
index = 7
print("True label of datapoint =", int(y_train[index]))
print("Predicted label of datapoint =", int(best_clusters.labels_[index]))
print("As image:\n")
plt.imshow(X_train[index].reshape(28,28), cmap=plt.cm.gray_r)
plt.show()

**(b)** Using visualization techniques analogous to what we have done in the Clustering notebook
   for the faces data, can you determine the 'nature' of the 10 constructed clusters?
   Do the clusters (roughly) coincide with the 10 different actual digits?

In [ ]:
fig,axes = plt.subplots(2,5, figsize=(15, 8))
for ax,cc,i in zip (axes.ravel(),best_clusters.cluster_centers_,np.arange(axes.ravel().size)):
    ax.set_title("digit: {}, size: {}".format(i,len(np.where(best_clusters.labels_==i)[0])))
    selected_pic = cc
    full_pic = []
    for j in range(0, 784):
        if(selection_mask[j]):
            full_pic.append(float(selected_pic[0]))
            selected_pic = selected_pic[1:]
        else:
            full_pic.append(float(0))   
    ax.imshow(np.array(full_pic).reshape(28,28),cmap=plt.cm.gray_r)
plt.show()

In [ ]:
distances = best_clusters.transform(selected_features)

pseudo_medoids=np.argmin(distances,axis=0)
fig,axes = plt.subplots(2,5,figsize=(15,8))
for ax,pm,i in zip (axes.ravel(),pseudo_medoids,np.arange(axes.ravel().size)):
    ax.set_title(i)
    ax.imshow(X_train[pm].reshape(28,28),cmap=plt.cm.gray_r)
plt.show()

**(c)** Perform a supervised clustering evaluation using adjusted rand index.
   Are the results stable, when you perform several random restarts of k-means?

In [ ]:
for rand_index in range(1,10):
    clusters = KMeans(n_clusters=10, init='random', n_init=10, random_state = rand_index).fit(X_train)
    labels = clusters.labels_
    accuracy = accuracy_score(y_train, labels);
    print(
        "For rand_index =",
        rand_index,
        "; accuracy = ",
        accuracy
    )

**(d)** Now perform hierarchical clustering on the data.
   (in order to improve visibility in the constructed dendrograms, you can also use a
   much reduced dataset as constructed using sklearn.utils.resample shown below).
   Does the visual analysis of the dendrogram indicate a natural number of clusters?

In [ ]:
small_mnist_data, small_mnist_label = utils.resample(mnist_data, mnist_label, n_samples=200,replace='false')

linkage_data = linkage(small_mnist_data, method='ward', metric='euclidean')
dendrogram(linkage_data)

plt.show()

**(e)** Using different cluster distance metrics (ward,single,average, etc.),
   what do the clusterings look like that are produced at the level of k=10 clusters?
   See the Clustering notebook for the needed Python code, including the fcluster
   method to retrieve 'plain' clusterings from the hierarchical clustering.

In [ ]:
small_mnist_data,small_mnist_label = skl.utils.resample(mnist.data,mnist.target,n_samples=200,replace='false')

**(f)** Do a DBSCAN clustering of the small dataset. Tweak the different parameters.

In [ ]:
def plotCluster(clusters, X_scaled):
    plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=clusters, s=60)
    plt.xlabel("Feature 0")
    plt.ylabel("Feature 1")
    print("Unique groups: {}".format(np.unique(clusters)))

In [ ]:
from sklearn.datasets import make_moons, make_blobs
from sklearn.cluster import DBSCAN

X = normalized_images
y = small_mnist_label
# rescale the data to zero mean and unit variance
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
# parameters
# epsilon - local radius for expanding clusters
# min_samples - the fewest number of points required to form a cluster
# unique groups show an array of unique formed groups. If array containing -1 number, means there is noise. 


In [ ]:
# DBSCAN clustering without parameters / default epsilon is 0.5 and min_samples are 4
dbscan = DBSCAN()
clusters = dbscan.fit_predict(X_scaled)
# plot the cluster assignments
plotCluster(clusters, X_scaled)


In [ ]:
# Epsilon 0.5 is too big. So it's forming only one group 
# DBSCAN clustering with eps 0.3 and default min_samples = 4
dbscan = DBSCAN(eps = 0.3)
clusters = dbscan.fit_predict(X_scaled)
# plot the cluster assignments
plotCluster(clusters, X_scaled)

In [ ]:
# Now it's possible to see two groups and one sample of noise. 

In [ ]:
# DBSCAN clustering with eps 0.4 and min_samples = 3
dbscan = DBSCAN(eps = 0.4, min_samples = 3)
clusters = dbscan.fit_predict(X_scaled)
# plot the cluster assignments
plotCluster(clusters, X_scaled)

In [ ]:
# With eps 0.4 and min_sample 3 we can see two groups

In [ ]:
# DBSCAN clustering with eps 0.01 and min_samples = 1
dbscan = DBSCAN(eps = 0.01, min_samples = 1)
clusters = dbscan.fit_predict(X_scaled)
# plot the cluster assignments
plotCluster(clusters, X_scaled)

In [ ]:
X, y = make_blobs(random_state=0, n_samples=100)

# rescale the data to zero mean and unit variance
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
# With low eps 0.01 and minimum sample to form group we can see that each sample form it's own group

In [ ]:
# DBSCAN clustering with default parameters on blob map
dbscan = DBSCAN()
clusters = dbscan.fit_predict(X_scaled)
# plot the cluster assignments
plotCluster(clusters, X_scaled)

In [ ]:
# In this example we can see 2 groups and noise.

In [ ]:
# DBSCAN clustering with eps 0.2
dbscan = DBSCAN(eps = 0.2)
clusters = dbscan.fit_predict(X_scaled)
# plot the cluster assignments
plotCluster(clusters, X_scaled)

In [ ]:
# After decresing eps value we have two small groups and a lot of noise areound. Epsilon is to small so the radius distance is not enough to form groups.

In [ ]:
# DBSCAN clustering with eps 0.6
dbscan = DBSCAN(eps = 0.6)
clusters = dbscan.fit_predict(X_scaled)
# plot the cluster assignments
plotCluster(clusters, X_scaled)

In [ ]:
# After incresing epsilon, it is possible to see one big group with the noise around.

In [ ]:
# DBSCAN clustering with eps 0.6 and min_samples = 1
dbscan = DBSCAN(eps = 0.6, min_samples = 1)
clusters = dbscan.fit_predict(X_scaled)
# plot the cluster assignments
plotCluster(clusters, X_scaled)

In [ ]:
# Now I decreased number of samples to form group to one. Now we have 5 groups and no noise

**(g)** Try to compare the different clustering methods on the MNIST dataset in the same way
   the book does on the faces dataset on pp. 195-206.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN, AgglomerativeClustering
X = np.array(normalized_images)
labels = mnist_label

pca = PCA(n_components=100, whiten=True, random_state=0)
pca.fit_transform(X)
X_pca = pca.transform(X)

dbscan = DBSCAN()
labels = dbscan.fit_predict(X_pca)
print("Unique labels: {}".format(np.unique(labels)))

In [ ]:
dbscan = DBSCAN(min_samples=3)
labels = dbscan.fit_predict(X_pca)
print("Unique labels: {}".format(np.unique(labels)))

In [ ]:
dbscan = DBSCAN(min_samples=3, eps=13)
labels = dbscan.fit_predict(X_pca)
print("Unique labels: {}".format(np.unique(labels)))

In [ ]:
print("Number of points per cluster: {}".format(np.bincount(labels + 1)))

In [ ]:
noise = X[labels==-1]

fig, axes = plt.subplots(3, 9, subplot_kw={'xticks': (), 'yticks': ()},
 figsize=(12, 4))
for image, ax in zip(noise, axes.ravel()):
 ax.imshow(image.reshape(28,28),cmap='gray')

In [ ]:
for eps in [1, 3, 5, 7, 9, 11, 12]:
 print("\neps={}".format(eps))
 dbscan = DBSCAN(eps=eps, min_samples=3)
 labels = dbscan.fit_predict(X_pca)
 print("Clusters present: {}".format(np.unique(labels)))
 print("Cluster sizes: {}".format(np.bincount(labels + 1)))

In [ ]:
dbscan = DBSCAN(min_samples=3, eps=12)
labels = dbscan.fit_predict(X_pca)
for cluster in range(max(labels) + 1):
 mask = labels == cluster
 n_images = np.sum(mask)
 fig, axes = plt.subplots(1, n_images, figsize=(n_images * 1.5, 4),
 subplot_kw={'xticks': (), 'yticks': ()})
 for image, label, ax in zip(X_people[mask], y_people[mask], axes):
     ax.imshow(image.reshape(28,28), vmin=0, vmax=1)


In [ ]:
km = KMeans(n_clusters=10, random_state=0)
labels_km = km.fit_predict(X_pca)
print("Cluster sizes k-means: {}".format(np.bincount(labels_km)))


In [ ]:
fig, axes = plt.subplots(2, 5, subplot_kw={'xticks': (), 'yticks': ()},
 figsize=(12, 4))
for center, ax in zip(km.cluster_centers_, axes.ravel()):
 ax.imshow(pca.inverse_transform(center).reshape(28,28),
 vmin=0, vmax=1)


In [ ]:
from sklearn.cluster import AgglomerativeClustering
# extract clusters with ward agglomerative clustering
agglomerative = AgglomerativeClustering(n_clusters=10)
labels_agg = agglomerative.fit_predict(X_pca)
print("Cluster sizes agglomerative clustering: {}".format(
 np.bincount(labels_agg)))

## 4. The Local Elections

__You should be able to do this exercise after Lecture 6.__

In the local elections of 2021, around 100 candidates stood for election for the city council of Horsens. 83 of them represented a national party, had more than one candidate and provided answers to the <a href="https://www.dr.dk/nyheder/politik/kandidattest">DR Candidate Test</a>, a test designed to help voters find out who they should vote for. In this test, the candidates answered 18 questions, which we will use as features in the following. The politicians belong to 9 parties, which will be our classes.

The numpy files `X_Horsens.npy` and `Y_Horsens.npy` contains the data. `Y_Horsens.npy` contains a letter representing the party to which each candidate belongs. The following parties are represented:

| Party letter | Party name | Party name (English) | Political position | Party color |
| :-: | :-: | :-: | :-: | :-: |
| A | Socialdemokratiet | Social Democrats | Centre-left | Red |
| B | Radikale Venstre | Social Liberal Party | Centre-left | Indigo |
| C | Det Konservative Folkeparti | Conservative People's Party | Right-wing | Green |
| D | Nye Borgerlige | New Right | Far-right | Black |
| F | Socialistisk Folkeparti | Socialist People's Party | Left-wing | Fuchsia |
| I | Liberal Alliance | Liberal Alliance | Right-wing | Cyan |
| O | Dansk Folkeparti | Danish People's Party | Far-right | Yellow |
| V | Venstre | Danish Liberal Party | Centre-right | Blue |
| Z* | Enhedslisten | Red-Green Alliance | Far-left | Dark red |

*_Note that, although the party letter of Enhedslisten is actually Ø, we will here use Z to avoid any complications with the wonderful Danish letters Æ, Ø and Å. Feel free to change the Z back to an Ø if you find that it does not cause any problems._

Meanwhile, `X_Horsens.npy` contains the answers to the test as numbers between -1.5 and 1.5, such that -1.5 is "Strongly disagree", -0.5 is "Disagree", 0.5 is "Agree" and 1.5 is "Strongly agree". The 18 questions concern, in order, subdivision, schools, windmills, building permits, tall buildings, housing, child care, culture, nursing homes, taxes, sports, refugees, nursing homes (again), public transportation, meat-free days, welfare, privatization, and religious minorities.

Both files can be imported using `numpy.load`.

__(a)__ How well do you (intuitively) expect that we can predict the partisan affiliation of a candidate based on their answers to the test?

Julia  
It's supervised classification with 9 classes. Model might have problem classifing politicians from parties ideologically simillar. Also dataset isn't big, with 100 entries and 9 classes there might be not enough of data for model to learn well enough. It might be around 50-60%. 

__(b)__ Based on the answers from all 83 candidates for the Horsens city council, perform a Principal Component Analysis with 2 principal components. Plot the results in a figure using these 2 components as the axes. Label the points with the party letter and the appropriate color.

In [ ]:
#Julia
X_Horsens = np.load('datasets/4/X_Horsens.npy')
Y_Horsens = np.load('datasets/4/Y_Horsens.npy')


In [ ]:
#Julia
# build a PCA model
pca = decomposition.PCA(n_components=2)
pca.fit(X_Horsens)
# transform the digits data onto the first two principal components
parties_pca = pca.transform(X_Horsens)
colors = { "A":"#FF0707","B":"#4B0082","C":"#00FF00","D":"#000000","F" :"#FF00FF",
          "I":"#00FFFF","O":"#FFFF00","V":"#0000FF", "Z":"#8B0000"}


plt.figure(figsize=(10, 10))
plt.xlim(parties_pca[:, 0].min(), parties_pca[:, 0].max())
plt.ylim(parties_pca[:, 1].min(), parties_pca[:, 1].max())
for i in range(len(X_Horsens)):
    plt.text(parties_pca[i, 0], parties_pca[i, 1], str(Y_Horsens[i]),
             color = colors[Y_Horsens[i]],
             fontdict={'weight': 'bold', 'size': 9})
plt.xlabel("First principal component")
plt.ylabel("Second principal component")
print(pca.explained_variance_ratio_)

__(c)__ Comment on the results. You may consider the following questions for inspiration: Can the political parties be separated? Can the typical distinction of "left-wing" and "right-wing" be discerned? Which of the 18 questions (features) are most important?

In [ ]:
#Julia
#to answer first two questions we will plot left-wing, right-wing and centrist parties with colours: green, blue, yellow
#only rigth and left
colors = { "A":"#00FF00","B":"#00FF00","C":"#0000FF","D":"#0000FF","F" :"#00FF00","I":"#0000FF","O":"#0000FF","V":"#0000FF", "Z":"#00FF00"}

plt.figure(figsize=(10, 10))
plt.xlim(parties_pca[:, 0].min(), parties_pca[:, 0].max())
plt.ylim(parties_pca[:, 1].min(), parties_pca[:, 1].max())
for i in range(len(X_Horsens)):
    plt.text(parties_pca[i, 0], parties_pca[i, 1], str(Y_Horsens[i]),
             color = colors[Y_Horsens[i]],
             fontdict={'weight': 'bold', 'size': 9})
plt.xlabel("First principal component")
plt.ylabel("Second principal component")
print(pca.explained_variance_ratio_)

On above graph we can see how leftist(green) and right-wing(blue) parties were noticably seperated based on components.

In [ ]:
#right, left and center:
colors = { "A":"#FFFF00","B":"#FFFF00","C":"#0000FF","D":"#0000FF","F" :"#00FF00", "I":"#0000FF","O":"#0000FF","V":"#FFFF00", "Z":"#00FF00"}

plt.figure(figsize=(10, 10))
plt.xlim(parties_pca[:, 0].min(), parties_pca[:, 0].max())
plt.ylim(parties_pca[:, 1].min(), parties_pca[:, 1].max())
for i in range(len(X_Horsens)):
    plt.text(parties_pca[i, 0], parties_pca[i, 1], str(Y_Horsens[i]),
             color = colors[Y_Horsens[i]],
             fontdict={'weight': 'bold', 'size': 9})
plt.xlabel("First principal component")
plt.ylabel("Second principal component")
print(pca.explained_variance_ratio_)

Here we can see how the parties are seperated when we consider also centrist parties. We can see how they are seperated from left wing ones but mix with right-wing. 

In [ ]:
#To answer which questions are most 'important' I did pca with one component to see variance of each feature
Xs = StandardScaler().fit_transform(X_Horsens)
pca = decomposition.PCA(n_components=1)
res = pca.fit_transform(Xs)

print(pca.explained_variance_ratio_)#Percentage of variance explained by component.
print(pd.DataFrame(pca.components_,columns=list(range(0, 18)),index = ['PC-1']))

Looks like question number 10 and 14 are ones that are most important in this dataset.  
10. Det er en god idé, at dagpengesatsen for nyuddannede er sat ned  
14. Der bør dannes en regering hen over midten

In [ ]:
#Here just plotting percentage of explained variance depending on number of components(for curiosity only)
x = list(range(1, 1 + len(pca.explained_variance_ratio_)))
plt.plot(x, np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()


The number of candidates (83) is on the (very) low side when we want to do machine learning. Luckily, the neighbouring city of Databorg had no less than 8,300 candidates standing for election, with a political environment similar to that of Horsens. In the following, we will use the data from Databorg. These are stored in the numpy files `X_Databorg.npy` and `Y_Databorg.npy` in same format as the Horsens data.

__(d)__ Once again, perform a Principal Component Analysis and visualize the results. Compare the results to those of the Horsens data.

In [ ]:
#Julia
X_Databorg = np.load('datasets/4/X_Databorg.npy')
Y_Databorg = np.load('datasets/4/Y_Databorg.npy')

In [ ]:
# build a PCA model
pca = decomposition.PCA(n_components=2)
pca.fit(X_Databorg)
# transform the digits data onto the first two principal components
parties_pca = pca.transform(X_Databorg)
colors = { "A":"#FF0707","B":"#4B0082","C":"#00FF00","D":"#000000","F" :"#FF00FF",
          "I":"#00FFFF","O":"#FFFF00","V":"#0000FF", "Z":"#8B0000"}


plt.figure(figsize=(10, 10))
plt.xlim(parties_pca[:, 0].min(), parties_pca[:, 0].max())
plt.ylim(parties_pca[:, 1].min(), parties_pca[:, 1].max())
for i in range(len(X_Databorg)):
    plt.text(parties_pca[i, 0], parties_pca[i, 1], str(Y_Databorg[i]),
             color = colors[Y_Databorg[i]],
             fontdict={'weight': 'bold', 'size': 9})
plt.xlabel("First principal component")
plt.ylabel("Second principal component")
print(pca.explained_variance_ratio_)

In [ ]:
#to answer first two questions we will plot left-wing, right-wing and centrist parties with colours: green, blue, yellow
#only rigth and left
colors = { "A":"#00FF00","B":"#00FF00","C":"#0000FF","D":"#0000FF","F" :"#00FF00","I":"#0000FF","O":"#0000FF","V":"#0000FF", "Z":"#00FF00"}

plt.figure(figsize=(10, 10))
plt.xlim(parties_pca[:, 0].min(), parties_pca[:, 0].max())
plt.ylim(parties_pca[:, 1].min(), parties_pca[:, 1].max())
for i in range(len(X_Databorg)):
    plt.text(parties_pca[i, 0], parties_pca[i, 1], str(Y_Databorg[i]),
             color = colors[Y_Databorg[i]],
             fontdict={'weight': 'bold', 'size': 9})
plt.xlabel("First principal component")
plt.ylabel("Second principal component")
print(pca.explained_variance_ratio_)

In [ ]:
#right, left and center:
colors = { "A":"#FFFF00","B":"#FFFF00","C":"#0000FF","D":"#0000FF","F" :"#00FF00", "I":"#0000FF","O":"#0000FF","V":"#FFFF00", "Z":"#00FF00"}

plt.figure(figsize=(10, 10))
plt.xlim(parties_pca[:, 0].min(), parties_pca[:, 0].max())
plt.ylim(parties_pca[:, 1].min(), parties_pca[:, 1].max())
for i in range(len(X_Databorg)):
    plt.text(parties_pca[i, 0], parties_pca[i, 1], str(Y_Databorg[i]),
             color = colors[Y_Databorg[i]],
             fontdict={'weight': 'bold', 'size': 9})
plt.xlabel("First principal component")
plt.ylabel("Second principal component")
print(pca.explained_variance_ratio_)

Confident that we can predict the partisan affiliation of a politician reasonably well based on their answers to the test, we want to build a model that will allow us to distinguish between the 9 political parties. For this purpose, we split the data into a training and a validation set.

__(e)__ Split the data into a training and a validation set, with appropriate fractions.

In [ ]:
#Julia
X_train, X_test , y_train, y_test = train_test_split(X_Databorg, Y_Databorg,  random_state=69)

First, we assume that a Naive Bayes approach is sufficient for our purposes.

__(f)__ Comment on the basic assumption of the Naive Bayes approach. Is this a reasonable assumption for the problem at hand?

Julia  
Naive Byens assumes that given predictors are unrelated. So in this case it assumes that answer to question nr 1 is unrelated to answer to question nr 2 or any other. As it's not entirely true, for example welfare questions will be related, if one is answered negatively then it's highly probable that other one will be rated so as well. In our case we can try ignoring it, to check what results will it show

__(g)__ Classify the instances of the validation set using a Naive Bayes approach. Comment on the results.

In [ ]:
#Julia
nb = GaussianNB()
nb.fit(X_train, y_train)
print("Accuracy score train: {}".format(nb.score(X_train, y_train)))
print("Accuracy score test: {}".format(nb.score(X_test, y_test)))

Accuracy for test data is higher than I expected. Model is not overfitting.

Assume instead that a _k_-nearest neighbour approach is sufficient for our  needs.

__(h)__ Using default settings of the _k_-NN classifier, classify the instances of the validation set. Comment on the performance.

In [ ]:
#Julia
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print("Model accuracy on the test data: {}".format(knn.score(X_test,y_test)))
print("Model accuracy on the train data: {}".format(knn.score(X_train,y_train)))

Accuracy increased when using k-NN classifier. And it's just with default k. 

__(i)__ Play around with different values of _k_. Decide on a "good" value of _k_. Comment on the results.

In [ ]:
#Julia
training_accuracy = []
test_accuracy = []

ks = range(1, 30)

for k in ks:
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(X_train, y_train)
    # record training set accuracy
    training_accuracy.append(clf.score(X_train, y_train))
    # record generalization accuracy
    test_accuracy.append(clf.score(X_test, y_test))
    
plt.plot(ks, training_accuracy, label="training accuracy")
plt.plot(ks, test_accuracy, label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_neighbors")
plt.legend();

####  Looks like 15 neithbours would be optimal number in this case to avoid overfitting and get reasonable accuracy 


In [ ]:
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train, y_train)
print("Model accuracy on the test data: {}".format(knn.score(X_test,y_test)))
print("Model accuracy on the train data: {}".format(knn.score(X_train,y_train)))

We now try to use a decision tree instead.

__(j)__ What is the _minimum_ depth of an appropriate decision tree? Why?

Nikita  
The minimum depth is the number of nodes along the shortest path from the root node down to the nearest leaf node.

__(k)__ Build a decision tree with at least the depth from above. Play around with the tree depth. Include a figure that shows some relevant measure of the performance as a function of the tree depth. Comment on any issues of over-fitting. Decide on a tree which you will keep for later use. Can you do better than the _k_-NN classifier?

In [ ]:
#Nikita
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

import graphviz
from sklearn.tree import export_graphviz
import pydotplus

X_Databorg = np.load('datasets/4/X_Databorg.npy')
Y_Databorg = np.load('datasets/4/Y_Databorg.npy')
X_Horsens = np.load('datasets/4/X_Horsens.npy')
Y_Horsens = np.load('datasets/4/Y_Horsens.npy')



In [ ]:

X_train, X_test , y_train, y_test = train_test_split(X_Databorg, Y_Databorg, test_size=0.5, random_state=69)
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)


print("Accuracy on training data: {}".format(clf.score(X_train, y_train)))
print("Accuracy on testing data: {}".format(clf.score(X_test, y_test)))

## Example of overfitting

## Unpruned trees are therefore prone to overfitting and not generalizing well to new data



In [ ]:

## Lets do Pre-prunning
X_train, X_test , y_train, y_test = train_test_split(X_Databorg, Y_Databorg, test_size=0.5, random_state=69)

training_accuracy = []
test_accuracy = []

ks = range(1, 16)

for k in ks:
    clf = DecisionTreeClassifier(max_depth=k, random_state=0)
    clf.fit(X_train, y_train)
    # record training set accuracy
    training_accuracy.append(clf.score(X_train, y_train))
    # record generalization accuracy
    test_accuracy.append(clf.score(X_test, y_test))
    
plt.plot(ks, training_accuracy, label="training accuracy")
plt.plot(ks, test_accuracy, label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("max_depth")
plt.legend();


k-NN classifier shows better results than any decison tree with any depth

The best tree with max_depth 5. Trees with max_depth higher than 5-6 have issues with overfitting as training accurance is much higher than test accuracy.


__(l)__ What are the most important features? Visualize this in an appropriate way. Does it match what you would expect? Compare to the results of the PCA analysis. Do we expect them to be the same? Why/why not?

In [ ]:
#Nikita
print(clf.feature_importances_)

values = np.load('datasets/4/X_Databorg.npy')
keys = np.load('datasets/4/Y_Databorg.npy')

def plot_feature_importance(model):
    n_features = values.data.shape[1]
    plt.barh(np.arange(n_features), model, align='center')
    plt.yticks(np.arange(n_features))
    plt.xlabel('Feature importance')
    plt.ylabel("Feature")
    plt.ylim(-1, n_features)

plot_feature_importance(clf.feature_importances_)



In [ ]:

Xs = StandardScaler().fit_transform(values)
pca = decomposition.PCA(n_components=1)
res = pca.fit_transform(Xs)
pca.explained_variance_ratio_

print(pca.components_)
#plot_feature_importance(pca.components_)

We know that decision trees suffer from certain problems that may be solved by using decision forests.

__(m)__ Build a decision forest. Play around with the number of trees in the forest. Decide on a forest.

In [ ]:
#Nikita

from sklearn.ensemble import RandomForestClassifier

X_Databorg = np.load('datasets/4/X_Databorg.npy')
Y_Databorg = np.load('datasets/4/Y_Databorg.npy')

X_train, X_test , y_train, y_test = train_test_split(X_Databorg, Y_Databorg, test_size=0.5, random_state=69)

forest = RandomForestClassifier(n_estimators=33, random_state=2)
forest.fit(X_train, y_train)

print("Model accuracy on the original/test? data: {}".format(forest.score(X_test,y_test)))
print("Model accuracy on the train data: {}".format(forest.score(X_train,y_train)))


In [ ]:
training_accuracy = []
test_accuracy = []

ks = range(1, 50)

for k in ks:
    clf = RandomForestClassifier(n_estimators=k, random_state=0)
    clf.fit(X_train, y_train)
    print(k, clf.score(X_train, y_train), clf.score(X_test, y_test), clf.score(X_train, y_train) - clf.score(X_test, y_test))
    training_accuracy.append(clf.score(X_train, y_train))
    test_accuracy.append(clf.score(X_test, y_test))
   
    
plt.plot(ks, training_accuracy, label="training accuracy")
plt.plot(ks, test_accuracy, label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_estimators")
plt.legend();



__(n)__ Extract the most important features. Comment and compare with previously obtained results.

In [ ]:
#Nikita
plot_feature_importance(clf.feature_importances_)

Most important features in the single tree are 9, 16, 4

In [ ]:
plot_feature_importance(forest.feature_importances_)

Most important features in the forest are 9, 16, 4

In [ ]:
plot_feature_importance(clf.feature_importances_) ## tree - blue
plot_feature_importance(forest.feature_importances_) ## forest - orange


In [ ]:
plot_feature_importance(forest.feature_importances_) ## forest - blue
plot_feature_importance(clf.feature_importances_) ## tree - orange



Some of the features in the random forest become less important 



Finally, we want to compare the models we have worked with so far (i.e., Naive Bayes, _k_-NN, decision tree and decision forest).

__(o)__ Compare the results of the in terms of confusion matrices, accuracy, precision, recall, and f-score. How well can we predict the partisan affiliation of a candidate based on their answers to a test? How does this compare with your intuition? 

In [ ]:
#Julia
#I collected best versions of models we used and will try to compare them
tree = DecisionTreeClassifier(max_depth=6, random_state=0)
tree.fit(X_train, y_train)
forest = RandomForestClassifier(n_estimators=32, random_state=0)
forest.fit(X_train, y_train)

#=====================================================================
print('Naive Bayes')
print("Model accuracy: {}".format(nb.score(X_test,y_test)))
y_pred = nb.predict(X_test) 
print("Precision: {}".format(metrics.precision_score(y_test,y_pred, average='macro')))#What proportion of positive identifications was actually correct?
print("Recall: {}".format(metrics.recall_score(y_test,y_pred, average='macro')))#What proportion of actual positives was identified correctly?
print("F-score: {}".format(metrics.f1_score(y_test,y_pred, average='macro')))
cm = metrics.confusion_matrix(y_test,y_pred)
print(cm)
#=====================================================================
print('k-Neigbours classifier')
print("Model accuracy: {}".format(knn.score(X_test,y_test)))
y_pred = knn.predict(X_test) 
print("Precision: {}".format(metrics.precision_score(y_test,y_pred, average='macro')))
print("Recall: {}".format(metrics.recall_score(y_test,y_pred, average='macro')))
print("F-score: {}".format(metrics.f1_score(y_test,y_pred, average='macro')))
cm = metrics.confusion_matrix(y_test,y_pred)
print(cm)
#=====================================================================
print('Decision Tree Classifier')
print("Model accuracy: {}".format(tree.score(X_test,y_test)))
y_pred = tree.predict(X_test) 
print("Precision: {}".format(metrics.precision_score(y_test,y_pred, average='macro')))
print("Recall: {}".format(metrics.recall_score(y_test,y_pred, average='macro')))
print("F-score: {}".format(metrics.f1_score(y_test,y_pred, average='macro')))
cm = metrics.confusion_matrix(y_test,y_pred)
print(cm)
#=====================================================================
print('Random Forest Classifier')
print("Model accuracy: {}".format(forest.score(X_test,y_test)))
y_pred = forest.predict(X_test) 
print("Precision: {}".format(metrics.precision_score(y_test,y_pred, average='macro')))
print("Recall: {}".format(metrics.recall_score(y_test,y_pred, average='macro')))
print("F-score: {}".format(metrics.f1_score(y_test,y_pred, average='macro')))
cm = metrics.confusion_matrix(y_test,y_pred)
print(cm)

k-Neighbours classifier turned out to be the best in terms of accuracy and other metrics. We can predict kandidate's partisan affiliation with 0.82 accuracy what is way more than what I assumed in beginning.

## 5. Sentiment Analysis

__You should be able to do this exercise after Lecture 8.__

In this exercise we use the IMDb-dataset, which we will use to perform a sentiment analysis. The code below assumes that the data is placed in the same folder as this notebook. We see that the reviews are loaded as a pandas dataframe, and print the beginning of the first few reviews.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers

reviews = pd.read_csv('datasets/5/reviews.txt', header=None)
labels = pd.read_csv('datasets/5/labels.txt', header=None)
Y = (labels=='positive').astype(np.int_)

print(type(reviews))
print(reviews.head())

#removing reviews that are not strings

print(reviews.shape)

#reviews = reviews[~reviews[0].str.isnumeric()]
reviews = reviews.rename(columns={0 : "text"})
print(reviews.head())
#reviews.columns

**(a)** Split the reviews and labels in test, train and validation sets. The train and validation sets will be used to train your model and tune hyperparameters, the test set will be saved for testing.

In [ ]:
X_trainval, X_test, y_trainval, y_test = train_test_split(reviews, Y, random_state=69)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, random_state=69)
print("Size of training set:{}".format(X_train.shape[0]))
print("Size of validation set:{}".format(X_val.shape[0]))
print("Size of test set:{}".format(X_test.shape[0]))

**(b)** Use the `CountVectorizer` from `sklearn.feature_extraction.text` to create a Bag-of-Words representation of the reviews. (See an example of how to do this in chapter 7 of "Muller and Guido"). Only use the 10,000 most frequent words (use the `max_features`-parameter of `CountVectorizer`).

In [ ]:
vect = CountVectorizer(max_features = 10000)
vect.fit(X_train['text'])

X_train = vect.transform(X_train['text'])
X_test = vect.transform(X_test['text'])
X_val = vect.transform(X_val['text'])

X_train = pd.DataFrame(X_train.toarray(),columns = vect.get_feature_names())
X_val = pd.DataFrame(X_val.toarray(),columns = vect.get_feature_names())
X_test = pd.DataFrame(X_test.toarray(),columns = vect.get_feature_names())

**(c)** Explore the representation of the reviews. How is a single word represented? How about a whole review?

In [ ]:
print(X_train.head())
print(X_test.head())
print(X_val.head())

Single word is a feature that is if present in review has value 1 and if not then 0. Number of words was limited to 10000 most common words present in training set. Therefore one review has 10000 features now.  

**(d)** Train a neural network with a single hidden layer on the dataset, tuning the relevant hyperparameters to optimize accuracy. 

In [ ]:
X_train.shape[1]

In [ ]:
model = Sequential() #initialize a neural network model

input_dim = X_train.shape[1]
input_size = X_train.shape[0]

X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
X_val = np.asarray(X_val)
y_val = np.asarray(y_val)

model.add(Dense(10, activation='relu', input_dim=input_dim)) #hidden layer #kernel_regularizer = regularizers.l2(0.001)

#model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid')) #output layer

#sgd = optimizers.SGD(learning_rate = 0.1)

model.compile(loss='binary_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])

print(model.summary())

history = model.fit(X_train, y_train, epochs = 2, batch_size = 50, verbose = 0,validation_data=(X_val, y_val))#, validation_split = 0.2)


In [ ]:
history.history.keys()

In [ ]:
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
plot_history(history)

**(e)** Test your sentiment-classifier on the test set.

In [ ]:
X_test = np.array(X_test)
y_test = np.array(y_test)
print("Loss + accuracy on train data: {}".format(model.evaluate(X_train, y_train)))
print("Loss + accuracy on test data: {}".format(model.evaluate(X_test, y_test)))

**(h)** Use the classifier to classify a few sentences you write yourselves. 

In [ ]:
sentences = ["teribble movie, I loved it","Excellent movie, I hated it!","Awful movie, I regret I saw it","I will watch it again and again"]
#sentences = pd.DataFrame(sentences,columns = vect.get_feature_names())
sentences = vect.transform(sentences)

model.predict(sentences) 


In [ ]:
a = input()
for i in a:
    print(int(i)**2, end = '')

## 6. Speech Recognition

__You should be able to do this exercise after Lecture 9.__

In this exercise, we will work with the <a href="https://arxiv.org/pdf/1804.03209.pdf">Google Speech Command Dataset</a>, which can be downloaded from <a href="http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz">here</a> (note: you do not need to download the full dataset, but it will allow you to play around with the raw audiofiles). This dataset contains 105,829 one-second long audio files with utterances of 35 common words.

We will use a subset of this dataset as indicated in the table below.

| Word | How many? | Class # |
| :-: | :-: | :-: |
| Yes | 4,044 | 3 |
| No | 3,941 | 1 |
| Stop | 3,872 | 2 |
| Go | 3,880 | 0 |

The data is given in the files `XSound.npy` and `YSound.npy`, both of which can be imported using `numpy.load`. `XSound.npy` contains spectrograms (_e.g._, matrices with a time-axis and a frequency-axis of size 62 (time) x 65 (frequency)). `YSound.npy` contains the class number, as indicated in the table above.

__(a)__ Explore and prepare the data, including splitting the data in training, validation and testing data, handling outliers, perhaps taking logarithms, etc. Data preparation is - as always - quite important. Document what you do.

In [ ]:
pip install h5py
pip install typing-extensions
pip install wheel

In [ ]:
pip install tensorflow --no-cache-dir --force-reinstall

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from scipy.io.wavfile import read, write
from scipy import signal
from IPython.display import Audio
from numpy.fft import fft, ifft, fftfreq
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers


XSound = np.load('datasets/6/XSound.npy')[:500]
YSound = np.load('datasets/6/YSound.npy')[:500]

In [ ]:
classes = np.unique(YSound)
dataPoints = XSound.shape[0]
timeDim = XSound.shape[1]
freqDim = XSound.shape[2]
features = timeDim * freqDim

print(classes)

XSound_flat = XSound.reshape(-1)
minimum = np.min(XSound_flat)
maximum = np.max(XSound_flat)

print("min:",minimum,"\nmaxinum:", maximum)

In [ ]:
# Let's scale these values so they range between 0 and 1
def Scale(_min, _max, _val):
    return (_val - _min) / (_max - _min);

for i, X in enumerate(XSound):
    X_flat = X.reshape(timeDim * freqDim, -1)

    X_min = np.min(X_flat)
    X_max = np.max(X_flat)
    
    X_scaled = []
    for X_measure in X_flat:
        X_scaled.append(Scale(X_min, X_max, X_measure))
        
    XSound[i] = np.array(X_scaled).reshape(X.shape)

In [ ]:
XSound_flat = XSound.reshape(-1)
minimum = np.min(XSound_flat)
maximum = np.max(XSound_flat)

print("min:",minimum,"\nmaxinum:", maximum)

__(b)__ Visualize a few examples of yes's, no's, stop's and go's, so that you have a reasonable intuitive understanding of the difference between the words.

In [ ]:
idx = 1
sample = XSound[idx]

#display(sample)

plt.specgram(sample, fs = features)

plt.xlabel('Time')

plt.ylabel('Frequency') 

plt.show()

__(c)__ Train a neural network and at least one other algorithm on the data. Find a good set of hyperparameters for each model. Do you think a neural network is suitable for this kind of problem? Why/why not?

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation='tanh', input_shape=XSound[0].shape)) #32 feature maps, 3*3 local receptive fields
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units = 4, activation = 'softmax')) #fully connected output layer

sgd = optimizers.SGD(learning_rate = 0.1)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

history = model.fit(X_train, Y_train, epochs = 10, batch_size = 50, verbose = 1, validation_split = 0.3)

__(d)__ Classify instances of the validation set using your models. Comment on the results in terms of metrics you have learned in the course.

In [ ]:
model.predict(X_test, Y_test, epochs = 10, batch_size = 50, verbose = 1, validation_split = 0.3)
# o ex 4

__(e)__ Identify (a few) misclassified words, including what they are misclassified as. Visualize them as before, and compare with your intuitive understanding of how the words look. Do you find the misclassified examples surprising?

In [ ]:
#confusion matrix

## 7. Group Assignment & Presentation



__You should be able to start up on this exercise after Lecture 1.__

*This exercise must be a group effort. That means everyone must participate in the assignment.*

In this assignment you will solve a data science problem end-to-end, pretending to be recently hired data scientists in a company. To help you get started, we've prepared a checklist to guide you through the project. Here are the main steps that you will go through:

1. Frame the problem and look at the big picture
2. Get the data
3. Explore and visualise the data to gain insights
4. Prepare the data to better expose the underlying data patterns to machine learning algorithms
5. Explore many different models and short-list the best ones
6. Fine-tune your models
7. Present your solution 

In each step we list a set of questions that one should have in mind when undertaking a data science project. The list is not meant to be exhaustive, but does contain a selection of the most important questions to ask. We will be available to provide assistance with each of the steps, and will allocate some part of each lesson towards working on the projects.

Your group must submit a _**single**_ Jupyter notebook, structured in terms of the first 6 sections listed above (the seventh will be a video uploaded to some streaming platform, e.g. YouTube, Vimeo, etc.).

### 1. Analysis: Frame the problem and look at the big picture
1. Find a problem/task that everyone in the group finds interesting
2. Define the objective in business terms
3. How should you frame the problem (supervised/unsupervised etc.)?
4. How should performance be measured?

### 2. Get the data
1. Find and document where you can get the data from
2. Get the data
3. Check the size and type of data (time series, geographical etc)

### 3. Explore the data
1. Create a copy of the data for explorations (sampling it down to a manageable size if necessary)
2. Create a Jupyter notebook to keep a record of your data exploration
3. Study each feature and its characteristics:
    * Name
    * Type (categorical, int/float, bounded/unbounded, text, structured, etc)
    * Percentage of missing values
    * Check for outliers, rounding errors etc
4. For supervised learning tasks, identify the target(s)
5. Visualise the data
6. Study the correlations between features
7. Identify the promising transformations you may want to apply (e.g. convert skewed targets to normal via a log transformation)
8. Document what you have learned

### 4. Prepare the data
Notes:
* Work on copies of the data (keep the original dataset intact).
* Write functions for all data transformations you apply, for three reasons:
    * So you can easily prepare the data the next time you run your code
    * So you can apply these transformations in future projects
    * To clean and prepare the test set
    
    
1. Data cleaning:
    * Fix or remove outliers (or keep them)
    * Fill in missing values (e.g. with zero, mean, median, regression ...) or drop their rows (or columns)
2. Feature selection (optional):
    * Drop the features that provide no useful information for the task (e.g. a customer ID is usually useless for modelling).
3. Feature engineering, where appropriate:
    * Discretize continuous features
    * Use one-hot encoding if/when relevant
    * Add promising transformations of features (e.g. $\log(x)$, $\sqrt{x}$, $x^2$, etc)
    * Aggregate features into promising new features
4. Feature scaling: standardise or normalise features

### 5. Short-list promising models
We expect you to do some additional research and train at **least one model per team member**.

1. Train mainly quick and dirty models from different categories (e.g. linear, SVM, Random Forests etc) using default parameters
2. Measure and compare their performance
3. Analyse the most significant variables for each algorithm
4. Analyse the types of errors the models make
5. Have a quick round of feature selection and engineering if necessary
6. Have one or two more quick iterations of the five previous steps
7. Short-list the top three to five most promising models, preferring models that make different types of errors

### 6. Fine-tune the system
1. Fine-tune the hyperparameters
2. Once you are confident about your final model, measure its performance on the test set to estimate the generalisation error

### 7. Present your solution
1. Document what you have done
2. Create a nice 15 minute video presentation with slides
    * Make sure you highlight the big picture first
3. Explain why your solution achieves the business objective
4. Don't forget to present interesting points you noticed along the way:
    * Describe what worked and what did not
    * List your assumptions and you model's limitations
5. Ensure your key findings are communicated through nice visualisations or easy-to-remember statements (e.g. "the median income is the number-one predictor of housing prices")
6. Upload the presentation to some online platform, e.g. YouTube or Vimeo, and supply a link to the video in the notebook.

## References

Géron, A. 2017, *Hands-On Machine Learning with Scikit-Learn and Tensorflow*, Appendix B, O'Reilly Media, Inc., Sebastopol.
Wunderground.com. 2022. The Highest Anemometer-Measured Wind Speeds on Earth. Available at: <https://www.wunderground.com/cat6/the-highest-anemometer-measured-wind-speeds-on-earth#:~:text=253%20mph%20%E2%80%A2%20Barrow%20Island%2C%20Australia%20%E2%80%A2%20April%2010%2C%201996&text=Wind%20trace%20taken%20at%20Barrow,6%3A15%20pm%20local%20time.> [Accessed 11 October 2022].